## ERRANT for Mandarin GEC

### References
#### https://github.com/getao/errant2. 
#### https://www.aclweb.org/anthology/P17-1074/3. 
#### https://www.repository.cam.ac.uk/handle/1810/293719
#### https://github.com/ckiplab/ckiptagger (install and download model files)

# Using CkipTagger
中研院斷詞工具

In [3]:
# Visit https://github.com/ckiplab/ckiptagger to install and download model files
from ckiptagger import data_utils, construct_dictionary, WS, POS, NER
ws = WS('/Users/winniehsu/Program/ckipmodel/')
pos = POS('/Users/winniehsu/Program/ckipmodel/')

/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [4]:
import re
def before_edit(text):
    return re.sub('(\[\-|\-\]|\{\+.*?\+\})', '', text)
    
def after_edit(text):
    return re.sub('\[\-.*?\-\]|\{\+|\+\}', '', text)

text = '[-這-]也[-間-][-接-]{+直+}{+接+}{+透露+}{+出+}[-突-][-顯-][-出-]鴻海'

text_before = ' '.join(ws(before_edit(text).splitlines())[0])
print (text_before)

text_after = ' '.join(ws(after_edit(text).splitlines())[0])
print (text_after)

這 也 間接 突顯出 鴻海
也 直接 透露出 鴻海


# [Step 1] 加入詞性參考 by ckip
input
``` Python
text = '這也間接突顯出鴻海'
text = '也直接透露出鴻海'

```
return
``` Python
[('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
[('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]
```

In [481]:
def seg_pos(text):
    ##### Your code here #####

print (seg_pos(before_edit(text)))
print (seg_pos(after_edit(text)))

[('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
[('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]


In [473]:
sent_orig = seg_pos(before_edit(text))
sent_correct = seg_pos(after_edit(text)) 
sent_orig, sent_correct

([('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')],
 [('也', 'D'), ('透露出', 'VK'), ('鴻海', 'Nb')])

# [Strp 2] 計算 edit cost 返回cost_matrix, op_matrix
`Hint:[Strp 2]會呼叫[Strp 3]的方法，Cost可以自己調整`

```
M是Match
I是insert cost
D是delete cost
R是replace cost
I/D是insert和delete相加的cost
```
input
``` Python
orig = [('這', 'Nep'), ('也', 'D'), ('間接', 'D'), ('突顯出', 'VJ'), ('鴻海', 'Nb')]
cor = [('也', 'D'), ('直接', 'VH'), ('透露出', 'VK'), ('鴻海', 'Nb')]

```
return
``` Python
cost_matrix = 
 [[0, 2, 4, 5, 4],
  [4, 6, 8, 9, 8],
  [2, 0, 6, 7, 6],
  [3, 5, 5, 8, 7],
  [5, 7, 8, 5, 8],
  [4, 6, 8, 9, 0]]
    
op_matrix =
 [['O', 'I', 'I', 'I', 'I'],
  ['D', 'I/D', 'I/D', 'I/D', 'I/D'],
  ['D', 'M', 'I/D', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'I/D', 'I/D'],
  ['D', 'I/D', 'R', 'R', 'R'],
  ['D', 'I/D', 'I/D', 'I/D', 'M']]
```

In [487]:
def calculate_edit_cost(orig, cor):
    o_len = len(orig)
    c_len = len(cor)
    # Create the cost_matrix and the op_matrix
    cost_matrix = [[0 for j in range(c_len+1)] for i in range(o_len+1)]
    op_matrix = [["O" for j in range(c_len+1)] for i in range(o_len+1)]
    # Fill in the edges
    for i in range(1, o_len+1):
        cost_matrix[i][0] = nltk.edit_distance(orig[i-1][1] + orig[i-1][0], '')
        op_matrix[i][0] = "D"
    for j in range(1, c_len+1):
        cost_matrix[0][j] = nltk.edit_distance('', cor[j-1][1] + cor[j-1][0])
        op_matrix[0][j] = "I"

    # Loop through the cost_matrix
    for i in range(o_len):
        for j in range(c_len):
            # Matches
            if orig[i] == cor[j]:
                cost_matrix[i+1][j+1] = 0
                op_matrix[i+1][j+1] = "M"
            # Non-matches
            else:
                ##### Your code here #####
                
    # Return the matrices
    return cost_matrix, op_matrix

# [Strp 3] 計算Replace的cost
嘗試找出可能可以計算cost的方式，例如檢查詞性、詞的長度或詞的差異等，最後將所有的cost相加回傳。

`Hint 要給多少Cost可以自己設定`

input
``` Python
orig = ('突顯出', 'VJ')
cor = ('透露出', 'VK')

```
return cost

In [488]:
def get_sub_cost(o, c):
    if o == c: return 0
    ##### Your code here #####
    return cost

In [ ]:
calculate_edit_cost(sent_orig, sent_correct)

# [Step 4] format cost_matrix and op_matrix

return
``` Python
[(0, 'R', '這', '透露出'), (2, 'D', '間接'), (3, 'R', '突顯出', '直接')]
```

In [490]:
def format_align(cost_matrix, op_matrix, orig, cor):
    ##### Your code here #####

In [491]:
s = ws(before_edit(text).splitlines())[0]
t = ws(after_edit(text).splitlines())[0]
print(s)
print(t)
cost_matrix, op_matrix = calculate_edit_cost(sent_orig, sent_correct)
format_align(cost_matrix, op_matrix, s, t)

['這', '也', '間接', '突顯出', '鴻海']
['也', '直接', '透露出', '鴻海']


[(0, 'R', '這', '透露出'), (2, 'D', '間接'), (3, 'R', '突顯出', '直接')]